In [99]:
import mlbgame
import pybaseball
import numpy as np
import pandas as pd
import statsapi
import tqdm

In [100]:
sched = statsapi.get("schedule", {"sportId":1, "startDate":'2023-06-01', "endDate": '2023-06-03', "fields": "dates,date,games,gamePk,gameType,teams,away,home,team,id"})

In [143]:
season_sched_df = pd.DataFrame()
counter = 0
dates = sched['dates']
for date_idx in tqdm.tqdm(range(len(dates))):
    
    date = dates[date_idx]['date']
    games = dates[date_idx]['games']
    
    for game_idx in range(len(games)): # account for doubleheaders
        
        game = games[game_idx]
        gamePk = game['gamePk']
        gameType = game['gameType']
        teams = game['teams']
        away_team_id = teams['away']['team']['id']
        home_team_id = teams['home']['team']['id']
        away_team_name = statsapi.get('team', {'teamId':away_team_id})['teams'][0]['teamName']
        home_team_name = statsapi.get('team', {'teamId':home_team_id})['teams'][0]['teamName']
        away_team_location = statsapi.get('team', {'teamId':away_team_id})['teams'][0]['locationName']
        home_team_location = statsapi.get('team', {'teamId':home_team_id})['teams'][0]['locationName']
        
        season_sched_df.loc[counter, 'gamePk'] = int(gamePk)
        season_sched_df.loc[counter, 'gameType'] = gameType
        
        season_sched_df.loc[counter, 'away_team_location'] = away_team_location
        season_sched_df.loc[counter, 'away_team_name'] = away_team_name
        season_sched_df.loc[counter, 'away_team_id'] = int(away_team_id)
        
        season_sched_df.loc[counter, 'home_team_location'] = home_team_location
        season_sched_df.loc[counter, 'home_team_name'] = home_team_name
        season_sched_df.loc[counter, 'home_team_id'] = int(home_team_id)
        
        game_info = statsapi.boxscore_data(gamePk)
        runs_home = game_info['home']['teamStats']['batting']['runs']
        runs_away = game_info['away']['teamStats']['batting']['runs']
        
        season_sched_df.loc[counter, 'away_team_runs'] = runs_away
        season_sched_df.loc[counter, 'home_team_runs'] = runs_home
        
        
        counter += 1

100%|██████████| 3/3 [00:32<00:00, 10.69s/it]


In [145]:
game_plays_df = pd.DataFrame()
counter = 0
for game_idx in tqdm.tqdm(season_sched_df.index):
    
    gamePk = int(season_sched_df.loc[game_idx, 'gamePk'])
    game_allPlays = statsapi.get('game_playByPlay', {'gamePk': gamePk})['allPlays']
    
    for play_idx in range(len(game_allPlays)):
    
        play = game_allPlays[play_idx]
        play_about = play['about']
        play_halfInning = play_about['halfInning']
        play_inning = play_about['inning']

        play_result = play['result']
        play_result_type = play_result['type']
        play_result_event = play_result['event']
        play_result_isOut = play_result['isOut']

        play_matchup = play['matchup']
        play_batter_id = play_matchup['batter']['id']
        play_batter_fullName = play_matchup['batter']['fullName']
        play_batter_batSide = play_matchup['batSide']['code']
        play_pitcher_id = play_matchup['pitcher']['id']
        play_pitcher_fullName = play_matchup['pitcher']['fullName']
        play_pitcher_pitchHand = play_matchup['pitchHand']['code']

        play_splits_batter = play_matchup['splits']['batter']
        play_splits_pitcher = play_matchup['splits']['pitcher']
        play_splits_menOnBase = play_matchup['splits']['menOnBase']
        
        game_plays_df.loc[counter, 'gamePk'] = gamePk
        game_plays_df.loc[counter, 'play_halfInning'] = play_halfInning
        game_plays_df.loc[counter, 'play_inning'] = play_inning
        game_plays_df.loc[counter, 'play_result_type'] = play_result_type
        game_plays_df.loc[counter, 'play_result_event'] = play_result_event
        game_plays_df.loc[counter, 'play_result_isOut'] = play_result_isOut
        game_plays_df.loc[counter, 'play_batter_id'] = play_batter_id
        game_plays_df.loc[counter, 'play_batter_fullName'] = play_batter_fullName
        game_plays_df.loc[counter, 'play_batter_batSide'] = play_batter_batSide
        game_plays_df.loc[counter, 'play_pitcher_id'] = play_pitcher_id
        game_plays_df.loc[counter, 'play_pitcher_fullName'] = play_pitcher_fullName
        game_plays_df.loc[counter, 'play_pitcher_pitchHand'] = play_pitcher_pitchHand
        game_plays_df.loc[counter, 'play_splits_batter'] = play_splits_batter
        game_plays_df.loc[counter, 'play_splits_pitcher'] = play_splits_pitcher
        game_plays_df.loc[counter, 'play_splits_menOnBase'] = play_splits_menOnBase
        
        counter += 1

100%|██████████| 38/38 [00:27<00:00,  1.40it/s]


In [147]:
game_plays_df[game_plays_df['gamePk'] == 717901.0]

,gamePk,play_halfInning,play_inning,play_result_type,play_result_event,play_result_isOut,play_batter_id,play_batter_fullName,play_batter_batSide,play_pitcher_id,play_pitcher_fullName,play_pitcher_pitchHand,play_splits_batter,play_splits_pitcher,play_splits_menOnBase
2768,717901.0,top,1.0,atBat,Lineout,True,660670.0,Ronald Acuna Jr.,R,669194.0,Ryne Nelson,R,vs_RHP,vs_RHB,Empty
2769,717901.0,top,1.0,atBat,Flyout,True,621566.0,Matt Olson,L,669194.0,Ryne Nelson,R,vs_RHP,vs_LHB,Empty
2770,717901.0,top,1.0,atBat,Groundout,True,663586.0,Austin Riley,R,669194.0,Ryne Nelson,R,vs_RHP,vs_RHB,Empty
2771,717901.0,bottom,1.0,atBat,Lineout,True,656976.0,Pavin Smith,L,675911.0,Spencer Strider,R,vs_RHP,vs_LHB,Empty
2772,717901.0,bottom,1.0,atBat,Walk,False,606466.0,Ketel Marte,L,675911.0,Spencer Strider,R,vs_RHP,vs_LHB,Men_On
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2841,717901.0,top,9.0,atBat,Lineout,True,669221.0,Sean Murphy,R,656464.0,Kevin Ginkel,R,vs_RHP,vs_RHB,Empty
2842,717901.0,bottom,9.0,atBat,Flyout,True,668942.0,Josh Rojas,L,628452.0,Raisel Iglesias,R,vs_RHP,vs_LHB,Empty
2843,717901.0,bottom,9.0,atBat,Single,False,672515.0,Gabriel Moreno,R,628452.0,Raisel Iglesias,R,vs_RHP,vs_RHB,Men_On
2844,717901.0,bottom,9.0,atBat,Groundout,True,672695.0,Geraldo Perdomo,L,628452.0,Raisel Iglesias,R,vs_RHP,vs_LHB,RISP
